# Imports and Matching Files

In [1]:
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import csv

from astropy.io import fits
from astropy.coordinates import match_coordinates_sky as coords
from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u
from astropy.units import cds
cds.enable()

from astropy.cosmology import LambdaCDM, FlatLambdaCDM

from PIL import Image

from astropy.io import ascii
from astropy.io.ascii import masked
from astropy.table import Table

ValueError: 'Kelvin' did not parse as unit: At col 0, Kelvin is not a valid unit.  If this is meant to be a custom unit, define it with 'u.def_unit'. To have it recognized inside a file reader or other code, enable it with 'u.add_enabled_units'. For details, see https://docs.astropy.org/en/latest/units/combining_and_defining.html

In [ ]:
manga_file = fits.open(r"C:\Users\paiaa\Downloads\mnsa-0.3.0.fits")
spitzer_file = fits.open(r"C:\Users\paiaa\Downloads\asu (2).fit")
pipe3d_file = fits.open(r"C:\Users\paiaa\Downloads\SDSS17Pipe3D_v3_1_1.fits")

In [ ]:
hdu_manga = manga_file[2]
hdu_p3d = pipe3d_file[1]
hdu_spitzer = spitzer_file[1]

manga_data = hdu_manga.data
p3d_data = hdu_p3d.data
spitzer_data = hdu_spitzer.data

#hdu_spitzer.header

In [ ]:
matched_objs = pd.read_csv(r'C:\Users\paiaa\Downloads\Selected_Spitzer_Objects.csv')
matched_objs = matched_objs.drop('Unnamed: 0', axis = 1)
#matched_objs = matched_objs[matched_objs.filter]
#matched_objs

## SDSS model files

In [ ]:
AP04_spec = fits.open(r"C:\Users\paiaa\Blanton Lab\Models\SDSS models\kc-spec-0.3.1-AP04.fits")
AP06_spec = fits.open(r"C:\Users\paiaa\Blanton Lab\Models\SDSS models\kc-spec-0.3.1-AP06.fits")
AP08_spec = fits.open(r"C:\Users\paiaa\Blanton Lab\Models\SDSS models\kc-spec-0.3.1-AP08.fits")

AP04_params = fits.open(r"C:\Users\paiaa\Blanton Lab\Models\SDSS models\kc-params-0.3.1-AP04.fits")
AP06_params = fits.open(r"C:\Users\paiaa\Blanton Lab\Models\SDSS models\kc-params-0.3.1-AP06.fits")
AP08_params = fits.open(r"C:\Users\paiaa\Blanton Lab\Models\SDSS models\kc-params-0.3.1-AP08.fits")

mnsa3_1_file = fits.open(r"C:\Users\paiaa\Downloads\mnsa-0.3.1.fits")
mnsa3_1_AP04 = mnsa3_1_file[5].data
mnsa3_1_AP06 = mnsa3_1_file[7].data
mnsa3_1_AP08 = mnsa3_1_file[9].data

AP04_data = AP04_spec[1].data
AP06_data = AP06_spec[1].data
AP08_data = AP08_spec[1].data


# Getting Required Parameters to List with Spectra

In [ ]:
#AORkey and PlateIFU
aorkey = matched_objs['AORkey'].to_numpy()
plateifu = matched_objs['PlateIFU'].to_numpy()

#spitzer HDU containing PAH EW and silicate
spitz_hdu_3 = spitzer_file[3].data
manga_hdu_1 = manga_file[1].data
#W1, W2, W3 from Spitzer and Manga, PAH EW and Silicate from Spitzer and log_SFR from Pipe3D
W1_manga = np.array([])
W2_manga = np.array([])
W3_manga = np.array([])
W4_manga = np.array([])

log_SFR = np.array([])

W1_spitzer = np.array([])
W2_spitzer = np.array([])
W3_spitzer = np.array([])
W4_spitzer = np.array([])

PAH_EW = np.array([])

silicate = np.array([])

redshift = np.array([])

ra = np.array([])
dec = np.array([])

for i in range(aorkey.shape[0]):
    W1_manga = np.append(W1_manga, manga_data[manga_data['plateifu'] == plateifu[i]]['absmag'][:,5][0])
    W2_manga = np.append(W2_manga, manga_data[manga_data['plateifu'] == plateifu[i]]['absmag'][:,6][0])
    W3_manga = np.append(W3_manga, manga_data[manga_data['plateifu'] == plateifu[i]]['absmag'][:,7][0])
    W4_manga = np.append(W4_manga, manga_data[manga_data['plateifu'] == plateifu[i]]['absmag'][:,8][0])
    
    log_SFR = np.append(log_SFR, p3d_data[p3d_data['plateifu'] == plateifu[i]]['log_SFR_ssp'][0])
        
    W1_spitzer = np.append(W1_spitzer, spitzer_data[spitzer_data['aorkey'] == aorkey[i]]['W1mag'][0])
    W2_spitzer = np.append(W2_spitzer, spitzer_data[spitzer_data['aorkey'] == aorkey[i]]['W2mag'][0])
    W3_spitzer = np.append(W3_spitzer, spitzer_data[spitzer_data['aorkey'] == aorkey[i]]['W3mag'][0])
    W4_spitzer = np.append(W4_spitzer, spitzer_data[spitzer_data['aorkey'] == aorkey[i]]['W4mag'][0])
    
    PAH_EW = np.append(PAH_EW, spitz_hdu_3[spitz_hdu_3['aorkey'] == aorkey[i]]['EW6_2'][0])
    
    silicate = np.append(silicate, spitz_hdu_3[spitz_hdu_3['aorkey'] == aorkey[i]]['tau9_7'][0])
    
    redshift = np.append(redshift, spitzer_data[spitzer_data['aorkey'] == aorkey[i]]['z'][0])
    
    ra = np.append(ra, manga_hdu_1[manga_hdu_1['plateifu'] == plateifu[i]]['objra'][0])
    dec = np.append(dec, manga_hdu_1[manga_hdu_1['plateifu'] == plateifu[i]]['objdec'][0])
    
    #ra = np.append(ra, spitzer_data[spitzer_data['aorkey'] == aorkey[i]]['RAJ2000'][0])
    #dec = np.append(dec, spitzer_data[spitzer_data['aorkey'] == aorkey[i]]['DEJ2000'][0])
    #redshift = np.append(redshift, manga_data[manga_data['plateifu'] == plateifu[i]]['redshift'][0])
    

#calling model related data from MNSA 0.3.1
W1_AP04 = np.array([])
W2_AP04 = np.array([])
W3_AP04 = np.array([])
W4_AP04 = np.array([])

W1_AP06 = np.array([])
W2_AP06 = np.array([])
W3_AP06 = np.array([])
W4_AP06 = np.array([])

W1_AP08 = np.array([])
W2_AP08 = np.array([])
W3_AP08 = np.array([])
W4_AP08 = np.array([])

for i in range(aorkey.shape[0]):
    W1_AP04 = np.append(W1_AP04, mnsa3_1_AP04[mnsa3_1_AP04['plateifu'] == plateifu[i]]['absmag'][:,5][0])
    W2_AP04 = np.append(W2_AP04, mnsa3_1_AP04[mnsa3_1_AP04['plateifu'] == plateifu[i]]['absmag'][:,6][0])
    W3_AP04 = np.append(W3_AP04, mnsa3_1_AP04[mnsa3_1_AP04['plateifu'] == plateifu[i]]['absmag'][:,7][0])
    W4_AP04 = np.append(W4_AP04, mnsa3_1_AP04[mnsa3_1_AP04['plateifu'] == plateifu[i]]['absmag'][:,8][0])
    
    W1_AP06 = np.append(W1_AP06, mnsa3_1_AP06[mnsa3_1_AP06['plateifu'] == plateifu[i]]['absmag'][:,5][0])
    W2_AP06 = np.append(W2_AP06, mnsa3_1_AP06[mnsa3_1_AP06['plateifu'] == plateifu[i]]['absmag'][:,6][0])
    W3_AP06 = np.append(W3_AP06, mnsa3_1_AP06[mnsa3_1_AP06['plateifu'] == plateifu[i]]['absmag'][:,7][0])
    W4_AP06 = np.append(W4_AP06, mnsa3_1_AP06[mnsa3_1_AP06['plateifu'] == plateifu[i]]['absmag'][:,8][0])
    
    W1_AP08 = np.append(W1_AP08, mnsa3_1_AP08[mnsa3_1_AP08['plateifu'] == plateifu[i]]['absmag'][:,5][0])
    W2_AP08 = np.append(W2_AP08, mnsa3_1_AP08[mnsa3_1_AP08['plateifu'] == plateifu[i]]['absmag'][:,6][0])
    W3_AP08 = np.append(W3_AP08, mnsa3_1_AP08[mnsa3_1_AP08['plateifu'] == plateifu[i]]['absmag'][:,7][0])
    W4_AP08 = np.append(W4_AP08, mnsa3_1_AP08[mnsa3_1_AP08['plateifu'] == plateifu[i]]['absmag'][:,8][0])

In [ ]:
#dataframe with W1, W2, W3 from Spitzer and Manga, PAH EW and Silicate from Spitzer and log_SFR from Pipe3D
galaxy_data = pd.DataFrame()

galaxy_data['plateifu'] = plateifu
galaxy_data['aorkey'] = aorkey

galaxy_data['ra'] = ra
galaxy_data['dec'] = dec

galaxy_data['W1 manga'] = W1_manga
galaxy_data['W2 manga'] = W2_manga
galaxy_data['W3 manga'] = W3_manga
galaxy_data['W4 manga'] = W4_manga

galaxy_data['W1 spitzer'] = W1_spitzer
galaxy_data['W2 spitzer'] = W2_spitzer
galaxy_data['W3 spitzer'] = W3_spitzer
galaxy_data['W4 spitzer'] = W4_spitzer

galaxy_data['log SFR'] = log_SFR
galaxy_data['PAH EW'] = PAH_EW
galaxy_data['Silicate feature'] = silicate
galaxy_data['redshift'] = redshift
galaxy_data['W1-W2'] = W1_manga - W2_manga

#ordering dataframe by W1-W2
galaxy_data = galaxy_data.sort_values(by=['W1-W2'])

#creating dataframe for model data
model_data = pd.DataFrame()

model_data['plateifu'] = plateifu
model_data['aorkey'] = aorkey

model_data['W1 AP04'] = W1_AP04
model_data['W2 AP04'] = W2_AP04
model_data['W3 AP04'] = W3_AP04
model_data['W4 AP04'] = W4_AP04

model_data['W1 AP06'] = W1_AP06
model_data['W2 AP06'] = W2_AP06
model_data['W3 AP06'] = W3_AP06
model_data['W4 AP06'] = W4_AP06

model_data['W1 AP08'] = W1_AP08
model_data['W2 AP08'] = W2_AP08
model_data['W3 AP08'] = W3_AP08
model_data['W4 AP08'] = W4_AP08

model_data['W1-W2'] = W1_manga - W2_manga

#ordering model dataframe by W1-W2
model_data = model_data.sort_values(by=['W1-W2'])

In [ ]:
galaxy_data.to_csv('galaxy_data.csv')
#galaxy_data

In [ ]:
#ordering arrays by W1-W2
W1_manga = galaxy_data['W1 manga'].to_numpy()
W2_manga = galaxy_data['W2 manga'].to_numpy()
W3_manga = galaxy_data['W3 manga'].to_numpy()
W4_manga = galaxy_data['W4 manga'].to_numpy()

log_SFR = galaxy_data['log SFR'].to_numpy()

W1_spitzer = galaxy_data['W1 spitzer'].to_numpy()
W2_spitzer = galaxy_data['W2 spitzer'].to_numpy()
W3_spitzer = galaxy_data['W3 spitzer'].to_numpy()
W4_spitzer = galaxy_data['W4 spitzer'].to_numpy()

PAH_EW = galaxy_data['PAH EW'].to_numpy()

silicate = galaxy_data['Silicate feature'].to_numpy()

redshift = galaxy_data['redshift'].to_numpy()

plateifu = galaxy_data['plateifu'].to_numpy()
aorkey = galaxy_data['aorkey'].to_numpy()

W12 = galaxy_data['W1-W2'].to_numpy()

In [ ]:
#model_data

# Plotting Spitzer Spectra

In [ ]:
spectra = []
for i in range(aorkey.shape[0]):
    combined = []
    filepath = r"C:\Users\paiaa\Blanton Lab\corrected_low_res_all\\" + str(aorkey[i]) + ".csv"
    df = pd.read_csv(filepath)
    wavelength = df['wavelength'].to_numpy()
    flux = df['flux_jy'].to_numpy()
    flux_err = df['flux_jy_err'].to_numpy()

    combined = np.vstack((wavelength, flux, flux_err))

    spectra.append(combined)

In [ ]:

for i in range(plateifu.shape[0]):
    fig, ax = plt.subplots(ncols = 2, figsize = (12,5))
    axa, axb = ax 
    text = 'W1-W2: ' + "{:.3f}".format(W12[i]) + '\n\n' + 'SFR: ' + "{:.3f}".format(10**(log_SFR[i])) + ' $M_\odot$ $yr^{-1}$' + '\n\n' + 'PAH EW: '+ "{:.3f}".format(PAH_EW[i]) + ' $\mu$m''\n\n' + 'Silicate strength: ' + "{:.3f}".format(silicate[i]) + ' $erg$ $s^{-1}$'

    axa.plot(spectra[i][0], spectra[i][1], linewidth = 0.8, c = 'r')
    axb.text(0.2, 0.5,text , fontsize = 14, horizontalalignment='center',
         verticalalignment='center')

    axb.axis('off')
    axa.set_ylabel('flux density $f_\\nu$ [Jy]', fontsize = 12)
    axa.set_xlabel('wavelength [$\mu$m]', fontsize = 12)
    axa.set_ylim(np.array([- 0.05, 1.1]) * spectra[i][1].max())
    axa.set_title('plate-IFU: ' + str(plateifu[i]) + '   AORkey: ' + str(aorkey[i]), fontsize = 14)

    #plt.savefig(r"C:\Users\paiaa\Blanton Lab\Spitzer spectra\\" + str(plateifu[i]) + '_SED.jpg', dpi = 300)

In [ ]:
for i in range(plateifu.shape[0]):
    fig, ax = plt.subplots(ncols = 2, figsize = (12,5),  gridspec_kw={'width_ratios': [3, 1]})
    axa, axb = ax 
    text = 'W1-W2: ' + "{:.3f}".format(W12[i]) + '\n\n' + 'SFR: ' + "{:.3f}".format(10**(log_SFR[i])) + ' $M_\odot$ $yr^{-1}$' + '\n\n' + 'PAH EW: '+ "{:.3f}".format(PAH_EW[i]) + ' $\mu$m''\n\n' + 'Silicate strength: ' + "{:.3f}".format(silicate[i]) + ' $erg$ $s^{-1}$'

    line_wavelengths = np.array([6.985, 8.911, 10.511, 12.813,14.3, 15.555,18.713, 25.910,25.989,33.480,34.815])
    line_names = np.array(['[Ar II]','[Ar III]', '[S IV]','[NeII]','[NeV]','[NeIII]', '[S III]','[O IV]/[Fe II]','','[S III]','[Si II]'])
    #line_fwhm = np.array([0.053, 0.100, 0.100, 0.100, 0.140, 0.140, 0.340, 0.340, 0.340, 0.340])
    text_heights = np.array([0.25,0.25,0.25,0.25,0.25,0.28,0.2,0.2,0.25,0.2,0.2])
    h2s_wavelengths = np.array([5.511, 6.109, 6.909,  8.026, 9.665, 12.278,  17.035, 28.221])
    
    pah_wavelengths = np.array([5.27, 5.70, 6.22, 6.69, 7.42, 7.60, 7.85, 8.33, 8.61, 10.68, 11.23, 11.33, 11.99, 12.62, 12.69, 13.48, 14.04, 14.19, 15.90, 16.45, 17.04, 17.375, 17.87, 18.92, 33.10])
    #h2s_fwhm = np.array([0.053, 0.053, 0.053, 0.100, 0.100, 0.100,  0.140,  0.340])
    #marking line emission
    for j in range(line_wavelengths.shape[0]):
        #line_wavelengths = line_wavelengths*(1+redshift[j])
        if line_wavelengths[j] < spectra[i][0].max():
            
            x1 = line_wavelengths[j]*np.ones((100))
            #x1_fwhm = np.linspace(line_wavelengths[j]-line_fwhm[j]/2, line_wavelengths[j]+line_fwhm[j]/2, 100)
            
            line_index = np.where(spectra[i][0] == spectra[i][0][np.abs(spectra[i][0] - line_wavelengths[j]).argmin()])[0]
            #y1_fwhm = (spectra[i][1][line_index] + 0.08*spectra[i][1].max())*np.ones((100))
            
            axa.plot(x1, np.linspace(spectra[i][1][line_index] + 0.08*spectra[i][1].max(),spectra[i][1][line_index] + 0.18*spectra[i][1].max(),100) ,linewidth =0.75, c = 'navy')
            #axa.plot(x1_fwhm, y1_fwhm, linewidth =0.5, c = 'navy')
            
            axa.text(line_wavelengths[j], spectra[i][1][line_index] + text_heights[j]*spectra[i][1].max(), line_names[j], horizontalalignment='center', fontsize = 7.5, c = 'black')
        #marking H2S emission
    for k in range(h2s_wavelengths.shape[0]):
        if h2s_wavelengths[k] < spectra[i][0].max():
            x2 = h2s_wavelengths[k]*np.ones((100))
            #x2_fwhm = np.linspace(h2s_wavelengths[k]-h2s_fwhm[k]/2, h2s_wavelengths[k]+h2s_fwhm[k]/2, 100)
            
            h2s_index = np.where(spectra[i][0] == spectra[i][0][np.abs(spectra[i][0] - h2s_wavelengths[k]).argmin()])[0]
            #y2_fwhm = (spectra[i][1][h2s_index] + 0.08*spectra[i][1].max())*np.ones((100))
            
            axa.plot(x2,np.linspace(spectra[i][1][h2s_index] + 0.08*spectra[i][1].max(),spectra[i][1][h2s_index] + 0.13*spectra[i][1].max(),100) ,linewidth =0.5, c = 'darkgreen')
            #axa.plot(x2_fwhm, y2_fwhm, linewidth =0.5, c = 'darkgreen')
    
    #plotting pah lines
    for w in range(pah_wavelengths.shape[0]):
        if pah_wavelengths[w] < spectra[i][0].max():
            x3 = pah_wavelengths[w]*np.ones((100))
            pah_index = np.where(spectra[i][0] == spectra[i][0][np.abs(spectra[i][0] - pah_wavelengths[w]).argmin()])[0]
           
            axa.plot(x3, np.linspace(spectra[i][1][pah_index] + 0.08*spectra[i][1].max(), spectra[i][1][pah_index] + 0.13 * spectra[i][1].max(), 100), linewidth = 0.5, c = 'darkviolet')
    
    #plotting spectrum
    axa.plot(spectra[i][0], spectra[i][1], linewidth = 0.8, c = 'r')   
    axa.scatter(100, 100, c = 'darkgreen', label='H2S(7) - H2S(0)')
    axa.scatter(100, 100, c = 'darkviolet', label='PAH lines')
    axb.text(0.5, 0.5,text , fontsize = 14, horizontalalignment='center',
             verticalalignment='center')
    
    #axes parameters
    axb.axis('off')
    axa.set_ylabel('flux density $f_\\nu$ [Jy]', fontsize = 12)
    axa.set_xlabel('wavelength [$\mu$m]', fontsize = 12)
    
    axa.set_ylim(np.array([- 0.05, 1.25]) * spectra[i][1].max())
    axa.set_xlim(np.array([0.75*spectra[i][0].min(), 1.05*spectra[i][0].max()]))
    
    axa.legend(loc = 'upper left')
    axa.set_title('plate-IFU: ' + str(plateifu[i]) + '   AORkey: ' + str(aorkey[i]), fontsize = 14)
    #plt.savefig(r'C:\Users\paiaa\Blanton Lab\Spitzer spectra\SEDs_labelled\\' + str([i]) + '_SED.jpg', dpi = 300)

## Plotting W1, W2, W3 on spectra

In [ ]:
#computing luminosity distance to scale AB magnitudes
cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Tcmb0=2.725 * u.K, Om0=0.3)
luminosity_distance = cosmo.luminosity_distance(redshift)
luminosity_distance = luminosity_distance.to(u.pc)
abmag_dist = 10 * u.pc

In [ ]:
#converting from AB magnitudes to Janskys
W1_Jy = 3631 * (10**(-0.4 * W1_manga))
W2_Jy = 3631 * (10**(-0.4 * W2_manga))
W3_Jy = 3631 * (10**(-0.4 * W3_manga))
W4_Jy = 3631 * (10**(-0.4 * W4_manga))

W1234 = np.vstack((W1_Jy, W2_Jy, W3_Jy, W4_Jy))


In [ ]:
#scaling magnitudes
W1234 = W1234 * abmag_dist**2/luminosity_distance**2
#W1234 = np.vstack((W1_spitzer, W2_spitzer, W3_spitzer, W4_spitzer))
W1234

In [ ]:
wise_wavelengths = np.array([3.4, 4.6, 12, 22])

for i in range(plateifu.shape[0]):
    fig, ax = plt.subplots(ncols = 2, figsize = (12,5),  gridspec_kw={'width_ratios': [3, 1]})
    axa, axb = ax 
    text = 'W1-W2: ' + "{:.3f}".format(W12[i]) + '\n\n' + 'SFR: ' + "{:.3f}".format(10**(log_SFR[i])) + ' $M_\odot$ $yr^{-1}$' + '\n\n' + 'PAH EW: '+ "{:.3f}".format(PAH_EW[i]) + ' $\mu$m''\n\n' + 'Silicate strength: ' + "{:.3f}".format(silicate[i]) + ' $erg$ $s^{-1}$'

    line_wavelengths = np.array([6.985, 8.911, 10.511, 12.813,14.3, 15.555,18.713, 25.910,25.989,33.480,34.815])
    line_names = np.array(['[Ar II]','[Ar III]', '[S IV]','[NeII]','[NeV]','[NeIII]', '[S III]','[O IV]/[Fe II]','','[S III]','[Si II]'])
    #line_fwhm = np.array([0.053, 0.100, 0.100, 0.100, 0.140, 0.140, 0.340, 0.340, 0.340, 0.340])
    text_heights = np.array([0.25,0.25,0.25,0.25,0.25,0.28,0.2,0.2,0.25,0.2,0.2])
    h2s_wavelengths = np.array([5.511, 6.109, 6.909,  8.026, 9.665, 12.278,  17.035, 28.221])
    
    pah_wavelengths = np.array([5.27, 5.70, 6.22, 6.69, 7.42, 7.60, 7.85, 8.33, 8.61, 10.68, 11.23, 11.33, 11.99, 12.62, 12.69, 13.48, 14.04, 14.19, 15.90, 16.45, 17.04, 17.375, 17.87, 18.92, 33.10])
    
    #marking line emission
    for j in range(line_wavelengths.shape[0]):
        
        #line_wavelengths = line_wavelengths*(1+redshift[j])
        if line_wavelengths[j] < spectra[i][0].max():
            
            x1 = line_wavelengths[j]*np.ones((100))
            
            line_index = np.where(spectra[i][0] == spectra[i][0][np.abs(spectra[i][0] - line_wavelengths[j]).argmin()])[0]

            
            axa.plot(x1, np.linspace(spectra[i][1][line_index] + 0.08*spectra[i][1].max(),spectra[i][1][line_index] + 0.18*spectra[i][1].max(),100) ,linewidth =0.75, c = 'navy')
            axa.text(line_wavelengths[j], spectra[i][1][line_index] + text_heights[j]*spectra[i][1].max(), line_names[j], horizontalalignment='center', fontsize = 7.5, c = 'black')
        
    #plotting WISE bands
    for l in range(wise_wavelengths.shape[0]):
        if wise_wavelengths[l] < spectra[i][0].max():
            axa.scatter(wise_wavelengths[l], W1234[l][i], s = 15, color = 'blue')
    
    #marking H2S emission
    for k in range(h2s_wavelengths.shape[0]):
        if h2s_wavelengths[k] < spectra[i][0].max():
            x2 = h2s_wavelengths[k]*np.ones((100))
            #x2_fwhm = np.linspace(h2s_wavelengths[k]-h2s_fwhm[k]/2, h2s_wavelengths[k]+h2s_fwhm[k]/2, 100)
            
            h2s_index = np.where(spectra[i][0] == spectra[i][0][np.abs(spectra[i][0] - h2s_wavelengths[k]).argmin()])[0]
            #y2_fwhm = (spectra[i][1][h2s_index] + 0.08*spectra[i][1].max())*np.ones((100))
            
            axa.plot(x2,np.linspace(spectra[i][1][h2s_index] + 0.08*spectra[i][1].max(),spectra[i][1][h2s_index] + 0.13*spectra[i][1].max(),100) ,linewidth =0.5, c = 'darkgreen')
            #axa.plot(x2_fwhm, y2_fwhm, linewidth =0.5, c = 'darkgreen')
    
    #plotting pah lines
    for w in range(pah_wavelengths.shape[0]):
        if pah_wavelengths[w] < spectra[i][0].max():
            x3 = pah_wavelengths[w]*np.ones((100))
            pah_index = np.where(spectra[i][0] == spectra[i][0][np.abs(spectra[i][0] - pah_wavelengths[w]).argmin()])[0]
           
            axa.plot(x3, np.linspace(spectra[i][1][pah_index] + 0.08*spectra[i][1].max(), spectra[i][1][pah_index] + 0.13 * spectra[i][1].max(), 100), linewidth = 0.5, c = 'darkviolet')
    
    #plotting spectrum
    axa.plot(spectra[i][0], spectra[i][1], linewidth = 0.8, c = 'r')
    axa.plot()
    #axa.scatter(100, 100, c = 'darkgreen', label='H2S(7) - H2S(0)')
    #axa.scatter(100, 100, c = 'darkviolet', label='PAH lines')
    #axa.scatter(100, 100, color = 'blue', label = 'WISE W2, W3, W4')
    axb.text(0.5, 0.5,text , fontsize = 14, horizontalalignment='center',
             verticalalignment='center')
    
    #axes parameters
    #axa.set_yscale('log')
    axb.axis('off')
    axa.set_ylabel('flux density $f_\\nu$ [Jy]', fontsize = 12)
    axa.set_xlabel('wavelength [$\mu$m]', fontsize = 12)
    axa.set_ylim(np.array([- 0.05, 1.25]) * spectra[i][1].max())
    axa.set_xlim(np.array([0.75*spectra[i][0].min(), 1.05*spectra[i][0].max()]))
    
    #axa.legend(loc = 'upper left')
    axa.set_title('plate-IFU: ' + str(plateifu[i]) + '   AORkey: ' + str(aorkey[i]), fontsize = 14)
    
    #plt.savefig(r'C:\Users\paiaa\Blanton Lab\Spitzer spectra\SEDs_labelled\\' + str(plateifu[i]) + '_SED.jpg', dpi = 300)

## Compare to Models

In [ ]:
#finding indices of selected galaxies in MNSA file
index = np.array([])  

for i in range(plateifu.shape[0]):
    for j in range(manga_data.shape[0]):
        if mnsa3_1_AP04['plateifu'][j] == plateifu[i]:
            index = np.append(index, j) 
index

In [ ]:
W1_AP04_test = np.array([])
for i in range(index.shape[0]):
    W1_AP04_test = np.append(W1_AP04_test, mnsa3_1_AP04['absmag'][:,5][int(index[i])])

In [ ]:
#getting model data of W1, W2, W3, W4
#converting from AB magnitudes to Janskys
W1_Jy_AP04 = 3631 * (10**(-0.4 * model_data['W1 AP04'].to_numpy()))
W2_Jy_AP04 = 3631 * (10**(-0.4 * model_data['W2 AP04'].to_numpy()))
W3_Jy_AP04 = 3631 * (10**(-0.4 * model_data['W3 AP04'].to_numpy()))
W4_Jy_AP04 = 3631 * (10**(-0.4 * model_data['W4 AP04'].to_numpy()))

W1234_AP04 = np.vstack((W1_Jy_AP04, W2_Jy_AP04, W3_Jy_AP04, W4_Jy_AP04))

#scaling magnitudes
W1234_AP04 = W1234_AP04 * abmag_dist**2/luminosity_distance**2

#converting from AB magnitudes to Janskys
W1_Jy_AP06 = 3631 * (10**(-0.4 * model_data['W1 AP06'].to_numpy()))
W2_Jy_AP06 = 3631 * (10**(-0.4 * model_data['W2 AP06'].to_numpy()))
W3_Jy_AP06 = 3631 * (10**(-0.4 * model_data['W3 AP06'].to_numpy()))
W4_Jy_AP06 = 3631 * (10**(-0.4 * model_data['W4 AP06'].to_numpy()))

W1234_AP06 = np.vstack((W1_Jy_AP06, W2_Jy_AP06, W3_Jy_AP06, W4_Jy_AP06))

#scaling magnitudes
W1234_AP06 = W1234_AP06 * abmag_dist**2/luminosity_distance**2

#converting from AB magnitudes to Janskys
W1_Jy_AP08 = 3631 * (10**(-0.4 * model_data['W1 AP08'].to_numpy()))
W2_Jy_AP08 = 3631 * (10**(-0.4 * model_data['W2 AP08'].to_numpy()))
W3_Jy_AP08 = 3631 * (10**(-0.4 * model_data['W3 AP08'].to_numpy()))
W4_Jy_AP08 = 3631 * (10**(-0.4 * model_data['W4 AP08'].to_numpy()))

W1234_AP08 = np.vstack((W1_Jy_AP08, W2_Jy_AP08, W3_Jy_AP08, W4_Jy_AP08))

#scaling magnitudes
W1234_AP08 = W1234_AP08 * abmag_dist**2/luminosity_distance**2

In [ ]:
#getting model spectra of selected galaxies, each spectrum has 5994 points
AP04 = np.array([])
AP06 = np.array([])
AP08 = np.array([])

for i in range(index.shape[0]):
    AP04 = np.append(AP04, AP04_data[int(index[i])])
    AP06 = np.append(AP06, AP06_data[int(index[i])])
    AP08 = np.append(AP08, AP08_data[int(index[i])])

AP04 = AP04.reshape(37,5994)
AP06 = AP06.reshape(37,5994)
AP08 = AP08.reshape(37,5994)


In [ ]:
#getting wavelengths (x-axis)
wav04 = AP04_spec[0].data
wav06 = AP06_spec[0].data
wav08 = AP08_spec[0].data

In [ ]:
#converting wavelengths to microns (from angstroms)
wav04 = wav04 * u.Angstrom
wav04 = wav04.to(u.micron)

wav06 = wav06 * u.Angstrom
wav06 = wav06.to(u.micron)

wav08 = wav08 * u.Angstrom
wav08 = wav08.to(u.micron)

In [ ]:
#modified c to microns/s
c_new = cds.c
c_new = c_new.to(u.micron/u.s)

In [ ]:
#sample code for for loop to convert from f_lambda to f_nu to Janskys

#combined = np.vstack((AP04_spec[0].data, AP04_data[int(index[1])]))
#combined = combined[:, combined[1] > spectra[0][0].min()]
#combined = combined[:, combined[1] < spectra[0][0].max()]
#combined[1] = combined[1]* (combined[0]**2 / (c_new))
#combined[1] = 10**(23) * combined[1]
#plt.plot(combined[0], combined[1]*combined[0])
#plt.yscale('log')
#plt.xscale('log')
#plt.ylim(10**(-14),)
#plt.xlim(10**2.5, 10**6)

In [ ]:
wise_wavelengths = np.array([3.4, 4.6, 12, 22])

for i in range(plateifu.shape[0]):
    fig, ax = plt.subplots(ncols = 2, figsize = (11,5),  gridspec_kw={'width_ratios': [3, 1]})
    axa, axb = ax 
    text = 'W1-W2: ' + "{:.3f}".format(galaxy_data['W1-W2'].to_numpy()[i]) + '\n\n' + 'SFR: ' + "{:.3f}".format(10**(galaxy_data['log SFR'].to_numpy()[i])) + ' $M_\odot$ $yr^{-1}$' + '\n\n' + 'PAH EW: '+ "{:.3f}".format(galaxy_data['PAH EW'].to_numpy()[i]) + ' $\mu$m''\n\n' + 'Silicate strength: ' + "{:.3f}".format(galaxy_data['Silicate feature'].to_numpy()[i]) + ' $erg$ $s^{-1}$'

    line_wavelengths = np.array([6.985, 8.911, 10.511, 12.813,14.3, 15.555,18.713, 25.910,25.989,33.480,34.815])
    line_names = np.array(['[Ar II]','[Ar III]', '[S IV]','[NeII]','[NeV]','[NeIII]', '[S III]','[O IV]/[Fe II]','','[S III]','[Si II]'])
    #line_fwhm = np.array([0.053, 0.100, 0.100, 0.100, 0.140, 0.140, 0.340, 0.340, 0.340, 0.340])
    text_heights = np.array([0.25,0.25,0.25,0.25,0.25,0.28,0.2,0.2,0.25,0.2,0.2])
    h2s_wavelengths = np.array([5.511, 6.109, 6.909,  8.026, 9.665, 12.278,  17.035, 28.221])
    
    pah_wavelengths = np.array([5.27, 5.70, 6.22, 6.69, 7.42, 7.60, 7.85, 8.33, 8.61, 10.68, 11.23, 11.33, 11.99, 12.62, 12.69, 13.48, 14.04, 14.19, 15.90, 16.45, 17.04, 17.375, 17.87, 18.92, 33.10])
    
    #marking line emission
    for j in range(line_wavelengths.shape[0]):
        
        #line_wavelengths = line_wavelengths*(1+redshift[j])
        if line_wavelengths[j] < spectra[i][0].max():
            
            x1 = line_wavelengths[j]*np.ones((100))
            
            line_index = np.where(spectra[i][0] == spectra[i][0][np.abs(spectra[i][0] - line_wavelengths[j]).argmin()])[0]

            
            axa.plot(x1, np.linspace(spectra[i][1][line_index] + 0.08*spectra[i][1].max(),spectra[i][1][line_index] + 0.18*spectra[i][1].max(),100) ,linewidth =0.75, c = 'black')
            axa.text(line_wavelengths[j], spectra[i][1][line_index] + text_heights[j]*spectra[i][1].max(), line_names[j], horizontalalignment='center', fontsize = 7.5, c = 'black')
        
    #plotting WISE bands from models
    for l in range(wise_wavelengths.shape[0]):
        if wise_wavelengths[l] < spectra[i][0].max():
            axa.scatter(wise_wavelengths[l], W1234_AP04[l][i], s = 15, color = 'dodgerblue')
            axa.scatter(wise_wavelengths[l], W1234_AP06[l][i], s = 15, color = 'darkcyan')
            axa.scatter(wise_wavelengths[l], W1234_AP08[l][i], s = 15, color = 'darkblue')
    
    #marking H2S emission
    for k in range(h2s_wavelengths.shape[0]):
        if h2s_wavelengths[k] < spectra[i][0].max():
            x2 = h2s_wavelengths[k]*np.ones((100))
            #x2_fwhm = np.linspace(h2s_wavelengths[k]-h2s_fwhm[k]/2, h2s_wavelengths[k]+h2s_fwhm[k]/2, 100)
            
            h2s_index = np.where(spectra[i][0] == spectra[i][0][np.abs(spectra[i][0] - h2s_wavelengths[k]).argmin()])[0]
            #y2_fwhm = (spectra[i][1][h2s_index] + 0.08*spectra[i][1].max())*np.ones((100))
            
            #axa.plot(x2,np.linspace(spectra[i][1][h2s_index] + 0.08*spectra[i][1].max(),spectra[i][1][h2s_index] + 0.13*spectra[i][1].max(),100) ,linewidth =0.5, c = 'darkgreen')
            #axa.plot(x2_fwhm, y2_fwhm, linewidth =0.5, c = 'darkgreen')
    
    #plotting pah lines
    for w in range(pah_wavelengths.shape[0]):
        if pah_wavelengths[w] < spectra[i][0].max():
            x3 = pah_wavelengths[w]*np.ones((100))
            pah_index = np.where(spectra[i][0] == spectra[i][0][np.abs(spectra[i][0] - pah_wavelengths[w]).argmin()])[0]
           
            #axa.plot(x3, np.linspace(spectra[i][1][pah_index] + 0.08*spectra[i][1].max(), spectra[i][1][pah_index] + 0.13 * spectra[i][1].max(), 100), linewidth = 0.5, c = 'darkviolet')
    
    #plotting SDSS models [model[0] = wavelength, model[1] = f_lambda] (selecting mid IR wavelengths)
    model04 = np.vstack((wav04.value, AP04[i] * (10 ** 4)))
    model04 = model04[:, model04[0] > spectra[i][0].min()]
    model04 = model04[:, model04[0] < spectra[i][0].max()]
    
    model06 = np.vstack((wav06.value, AP06[i] * (10 ** 4)))
    model06 = model06[:, model06[0] > spectra[i][0].min()]
    model06 = model06[:, model06[0] < spectra[i][0].max()]
    
    model08 = np.vstack((wav08.value, AP08[i] * (10 ** 4)))
    model08 = model08[:, model08[0] > spectra[i][0].min()]
    model08 = model08[:, model08[0] < spectra[i][0].max()]
    
    #converting from f_lambda to f_nu
    model04[1] = model04[1]* (model04[0]**2 / (c_new))
    
    model06[1] = model06[1]* (model06[0]**2 / (c_new))
    
    model08[1] = model08[1]* (model08[0]**2 / (c_new))
    
    #converting to Janskys
    model04[1] = 10**(23) * model04[1]
    
    model06[1] = 10**(23) * model06[1]
    
    model08[1] = 10**(23) * model08[1]
       
    #plotting spectrum and SDSS models
    axa.plot(spectra[i][0], spectra[i][1], linewidth = 0.8, c = 'r')
    axa.plot(model04[0], model04[1], linewidth = 0.8, c = 'dodgerblue', label = 'AP04')
    axa.plot(model06[0], model06[1], linewidth = 0.8, c = 'darkcyan', label = 'AP06')
    axa.plot(model08[0], model08[1], linewidth = 0.8, c = 'darkblue', label = 'AP08')
    #axa.scatter(100, 100, c = 'darkgreen', label='H2S(7) - H2S(0)')
    #axa.scatter(100, 100, c = 'darkviolet', label='PAH lines')
    #axa.scatter(100, 100, color = 'blue', label = 'WISE W2, W3, W4')
    axb.text(0.5, 0.5,text , fontsize = 14, horizontalalignment='center',
             verticalalignment='center')
    
    #axes parameters
    axa.set_yscale('log')
    axa.set_xscale('log')
    axb.axis('off')
    axa.set_ylabel(' log flux density $f_\\nu$ [Jy]', fontsize = 12)
    axa.set_xlabel('log wavelength [$\mu$m]', fontsize = 12)
    #axa.set_ylim(np.array([- 0.05, 1.25]) * spectra[i][1].max())
    axa.set_xlim(np.array([0.75*spectra[i][0].min(), 1.05*spectra[i][0].max()]))
    
    axa.legend(loc = 'upper left')
    axa.set_title('plate-IFU: ' + str(galaxy_data['plateifu'].to_numpy()[i]) + '   AORkey: ' + str(galaxy_data['aorkey'].to_numpy()[i]), fontsize = 14)
    
    #plt.savefig(r'C:\Users\paiaa\Blanton Lab\Spitzer spectra\SEDs_w_models\\' + str(galaxy_data['plateifu'].to_numpy()[i]) + '_SED.jpg', bbox_inches = 'tight', pad_inches = 0.2, dpi = 300)

## Make PDF of Spectra

In [ ]:
def make_pdf():
    image_names = galaxy_data['plateifu'].to_numpy() + '_SED.jpg'

    images = [
        Image.open(r'C:\Users\paiaa\Blanton Lab\Spitzer spectra\SEDs_w_models\\' + f)
        for f in image_names]

    pdf_path = r'C:\Users\paiaa\Blanton Lab\Spitzer spectra\SEDs_w_models\\SEDs_w_models.pdf'
    
    images[0].save(
        pdf_path, "PDF" ,resolution=100.0, save_all=True, append_images=images[1:])

#make_pdf()